In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (177 kB/s)
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [4]:

spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("my_table")

result = spark.sql("SELECT * FROM my_table")

result.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
column_names = df.columns
print(column_names)

['id', 'date', 'date_built', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view']


In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import year, round

# Execute SQL query to calculate average price for four-bedroom houses in each year
query = "SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price FROM my_table WHERE bedrooms = 4 GROUP BY YEAR(date)"

# Execute the query and store the result in a new DataFrame
fourbed_avgprice = spark.sql(query)

# Show the result
fourbed_avgprice.show()


+----+---------+
|year|avg_price|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

# Execute SQL query to calculate average price for 3-bedroom, 3-bathroom homes in each year
query = "SELECT date_built, ROUND(AVG(price), 2) AS avg_price FROM my_table WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built"

# Execute the query and store the result in a new DataFrame
threebedthreebath_avg_price = spark.sql(query)

# Show the result
threebedthreebath_avg_price.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2016|290555.07|
|      2012|293683.19|
|      2017|292676.79|
|      2014|290852.27|
|      2013|295962.27|
|      2011|291117.47|
|      2015| 288770.3|
|      2010|292859.62|
+----------+---------+



In [10]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

# Execute SQL query to calculate average price for the specified criteria in each year
query = "SELECT date_built, ROUND(AVG(price), 2) AS avg_price FROM my_table WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built"

# Execute the query and store the result in a new DataFrame
threebedbath2000_avgprice = spark.sql(query)

# Show the result
threebedbath2000_avgprice.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2016| 293965.1|
|      2012|307539.97|
|      2017|280317.58|
|      2014|298264.72|
|      2013|303676.79|
|      2011|276553.81|
|      2015|297609.97|
|      2010|285010.22|
+----------+---------+



In [11]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

import time

# Start the timer
start_time = time.time()

# Execute SQL query to calculate the average price and view rating for homes greater than or equal to $350,000
query = "SELECT ROUND(AVG(price), 2) AS avg_price, AVG(view) AS avg_view FROM my_table WHERE price >= 350000"

# Execute the query and store the result in a new DataFrame
viewrating = spark.sql(query)

# Show the result
viewrating.show()

# Calculate the runtime
runtime = time.time() - start_time
print("Runtime: %.2f seconds" % runtime)

+---------+--------+
|avg_price|avg_view|
+---------+--------+
|473796.26|  32.264|
+---------+--------+

Runtime: 0.88 seconds


In [12]:
# 7. Cache the the temporary table home_sales.

df.createOrReplaceTempView("home_sales")
spark.table("home_sales").cache()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [13]:
# 8. Check if the table is cached.

is_cached = spark.catalog.isCached("home_sales")
if is_cached:
    print("The table is cached.")
else:
    print("The table is not cached.")

The table is cached.


In [15]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Run the query on uncached data and record the runtime
uncached_start_time = time.time()
uncached_result = spark.sql("SELECT AVG(price) AS AveragePrice FROM home_sales WHERE price >= 350000")
uncached_runtime = time.time() - uncached_start_time
uncached_result.show()

# Cache the table
spark.catalog.cacheTable("home_sales")

# Run the query on cached data and record the runtime
cached_start_time = time.time()
cached_result = spark.sql("SELECT AVG(price) AS AveragePrice FROM home_sales WHERE price >= 350000")
cached_runtime = time.time() - cached_start_time
cached_result.show()

# Compare the runtimes
print("Uncached Runtime:", uncached_runtime)
print("Cached Runtime:", cached_runtime)

# Compare the runtimes
print("Cached runtime improvement:", uncached_runtime - cached_runtime)


+------------------+
|      AveragePrice|
+------------------+
|473796.26220224716|
+------------------+

+------------------+
|      AveragePrice|
+------------------+
|473796.26220224716|
+------------------+

Uncached Runtime: 0.033418893814086914
Cached Runtime: 0.031675100326538086
Cached runtime improvement: 0.0017437934875488281


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

df.write.mode("overwrite").partitionBy("date_built").parquet("output.parquet")


In [17]:
# 11. Read the formatted parquet data.

parquet_df = spark.read.format("parquet").load("output.parquet")


In [18]:
# 12. Create a temporary table for the parquet data.

parquet_df.createOrReplaceTempView("parquet_table")


In [19]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 

query = "SELECT view, ROUND(AVG(price), 2) AS AveragePrice FROM parquet_table WHERE price >= 350000 GROUP BY view"
result = spark.sql(query)
result.show()

# Determine the runtime and compare it to the cached version.

import time

start_time = time.time()

# Execute the query
result = spark.sql(query)
result.show()

end_time = time.time()
runtime = end_time - start_time
print("Runtime:", runtime, "seconds")


+----+------------+
|view|AveragePrice|
+----+------------+
|   7|   403005.77|
|  51|   788128.21|
|  15|    404673.3|
|  54|   798684.82|
|  11|   399548.12|
|  29|   397771.65|
|  69|   750537.94|
|  42|    396964.5|
|  73|   752861.18|
|  87|   1072285.2|
|  64|   767036.67|
|   3|    398867.6|
|  30|    397862.0|
|  34|   401419.75|
|  59|    791453.0|
|   8|   398592.71|
|  28|   402124.62|
|  22|   402022.68|
|  85|  1056336.74|
|  35|   401934.21|
+----+------------+
only showing top 20 rows

+----+------------+
|view|AveragePrice|
+----+------------+
|   7|   403005.77|
|  51|   788128.21|
|  15|    404673.3|
|  54|   798684.82|
|  11|   399548.12|
|  29|   397771.65|
|  69|   750537.94|
|  42|    396964.5|
|  73|   752861.18|
|  87|   1072285.2|
|  64|   767036.67|
|   3|    398867.6|
|  30|    397862.0|
|  34|   401419.75|
|  59|    791453.0|
|   8|   398592.71|
|  28|   402124.62|
|  22|   402022.68|
|  85|  1056336.74|
|  35|   401934.21|
+----+------------+
only showing t

In [20]:
# 14. Uncache the home_sales temporary table.

spark.catalog.uncacheTable("my_table")

In [21]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")
print(is_cached)

False
